In [13]:
import os
from collections import defaultdict

import numpy as np
import scipy

from setup_paths import *
import visu_common
import measurement

meas_type = 'latency'
base_dir = 'v1_O3'

all_meas_configs = {
    'direction': ['s', 'r'],
    'clkM7': [60, 120, 240, 480],
    'clkM4': [60, 120, 240],
    'repeat': [256],
    'datasize': ([1]
                + [16*x for x in range(1, 17)]
                + [512, 1024, 1536]
                + [1024*x for x in range(2, 16)] + [16376]),
    'mem': ['D1', 'D2', 'D3'],
    'cache': ['none', 'i', 'd', 'id'],
}

config_list = measurement.config_to_config_list(all_meas_configs)
meas_configs_list = measurement.group_config_except(config_list, ['datasize', 'clkM7', 'clkM4'])

identified_params = defaultdict(lambda: defaultdict(dict))
for meas_configs in meas_configs_list:
    meas_configs = {k:(v if isinstance(v, list) else [v])
                    for k, v in meas_configs.items()}
    y, y_config_list = measurement.get_and_calc_meas(meas_configs, meas_type, base_dir)
    # all column vectors
    sizes = np.asarray([x['datasize'] for x in y_config_list]).reshape(-1, 1)
    m7 = np.asarray([x['clkM7'] for x in y_config_list]).reshape(-1, 1)
    m4 = np.asarray([x['clkM4'] for x in y_config_list]).reshape(-1, 1)
    y = y[0, :] # vector of means
    K = np.hstack(
        (1/m7*np.ones((len(sizes), 1)), # const part for sending
        1/m7*sizes, # size dependent part for sending
        1/m4*sizes,
        1/m4*np.ones((len(sizes), 1))))

    res = scipy.optimize.lsq_linear(K, y, bounds=(0, np.inf))
    print(res.message)
    # direction under mem in the dict
    mem = meas_configs['mem'][0]
    cache = meas_configs['cache'][0]
    direction = meas_configs['direction'][0]
    identified_params[mem][cache][direction] = list(res.x)

The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.


In [16]:
import json

# Writing the result to file
filename = 'models_long.json'
model_path = os.path.join(MODELS_PATH, base_dir)
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, filename), 'w') as file:
    json.dump(identified_params, file, indent=4)

In [14]:
print(identified_params)

defaultdict(<function <lambda> at 0x000001F54119EC20>, {'D1': defaultdict(<class 'dict'>, {'d': {'r': [1395.1583985521722, 4.32422121090115, 18.810706799610244, 1752.960068497788], 's': [553.9780850793837, 0.006367677150922901, 16.08401851126504, 2213.290381468761]}, 'i': {'r': [1280.6924012633867, 3.443620000297283, 20.247574223287344, 1979.3045390537043], 's': [605.0118355056569, 5.123970769163083, 15.68901360948578, 2269.4293844908057]}, 'id': {'r': [1399.1252112358488, 4.329385154395634, 18.797748597878265, 1218.5940039394884], 's': [546.2727396730173, 0.008815207032876415, 16.078861762440706, 2014.652036643433]}, 'none': {'r': [1573.3040486006948, 3.2773823982188723, 20.68041937038953, 2249.072358575892], 's': [614.5814974605076, 5.159041977722122, 15.898404165327632, 2490.8572368064947]}}), 'D2': defaultdict(<class 'dict'>, {'d': {'r': [1376.4578308465052, 4.746890294171463, 12.762088903970692, 1577.2723379157503], 's': [575.6342176919075, 0.011209417195956917, 14.033855281073617